<a href="https://colab.research.google.com/github/programminghunk/sugarcane/blob/main/Yolov8_tinier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download ultralytics

---



In [1]:
!git clone https://github.com/ultralytics/ultralytics.git

Cloning into 'ultralytics'...
remote: Enumerating objects: 40065, done.
remote: Counting objects: 100% (754/754), done.
remote: Compressing objects: 100% (531/531), done.
remote: Total 40065 (delta 456), reused 455 (delta 222), pack-reused 39311 (from 1)
Receiving objects: 100% (40065/40065), 31.09 MiB | 16.95 MiB/s, done.
Resolving deltas: 100% (29522/29522), done.


Install ultralytics in edit mode/locally

---



In [2]:
%cd ultralytics
!pip install -e '.[dev]'

/content/ultralytics
Obtaining file:///content/ultralytics
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.6/51.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.6 MB/s eta 0:00:00
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 76.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.7/234.7 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 12.0 MB/s eta 0:00:00
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.5/153.5 kB 1

Check installation


In [3]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.0/112.6 GB disk)


In [4]:
%pwd

'/content/ultralytics'

In [5]:
%cd

/root


In [11]:
# prompt: change directory from root to content/ultralytics

%cd /content/ultralytics


/content/ultralytics


In [13]:
%cd ..

/content


# downlaod the dataset in content\datasets dir

In [14]:
!pip install roboflow --quiet

from roboflow import Roboflow
rf = Roboflow(api_key="y4uzPDgeB6rEDxdHhxsW")
project = rf.workspace("buddetection").project("np_mynp")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.2.90, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to np_mynp-2 in yolov8:: 100%|██████████| 6719/6719 [00:01<00:00, 4835.52it/s]


Write newconf.yaml to modify the backbone

```
# Parameters
nc: 2 # number of classes
scales: # model compound scaling constants, i.e. 'model=yolov8n.yaml' will call yolov8.yaml with scale 'n'
  # [depth, width, max_channels]
  n: [0.33, 0.25, 1024] # YOLOv8n summary: 225 layers,  3157200 parameters,  3157184 gradients,   8.9 GFLOPs
  s: [0.33, 0.50, 1024] # YOLOv8s summary: 225 layers, 11166560 parameters, 11166544 gradients,  28.8 GFLOPs
  m: [0.67, 0.75, 768] # YOLOv8m summary: 295 layers, 25902640 parameters, 25902624 gradients,  79.3 GFLOPs
  l: [1.00, 1.00, 512] # YOLOv8l summary: 365 layers, 43691520 parameters, 43691504 gradients, 165.7 GFLOPs
  x: [1.00, 1.25, 512] # YOLOv8x summary: 365 layers, 68229648 parameters, 68229632 gradients, 258.5 GFLOPs

# YOLOv8.0n backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]] # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]] # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]] # 3-P3/8
  - [-1, 1, RepNCSPELAN4, [64, 64, 32, 3]] # 4
  - [-1, 1, Conv, [128, 3, 2]] # 5-P4/16
  - [-1, 1, RepNCSPELAN4, [256, 128, 48, 3]] # 6
  - [-1, 1, Conv, [256, 3, 2]] # 7-P5/32
  - [-1, 1, RepNCSPELAN4, [256, 256, 64, 3]] # 8
  - [-1, 1, SPPF, [256, 5]] # 9

# YOLOv8.0n head
head:
  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 6], 1, Concat, [1]] # cat backbone P4
  - [-1, 3, C2f, [512]] # 12

  - [-1, 1, nn.Upsample, [None, 2, "nearest"]]
  - [[-1, 4], 1, Concat, [1]] # cat backbone P3
  - [-1, 3, C2f, [256]] # 15 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]] # cat head P4
  - [-1, 3, C2f, [512]] # 18 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]] # cat head P5
  - [-1, 3, C2f, [1024]] # 21 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]] # Detect(P3, P4, P5)

```



Summary of the new model

In [18]:
import ultralytics
from ultralytics import YOLO

model = YOLO("ultralytics/ultralytics/cfg/models/v8/newconf.yaml")

# Display model information (optional)
model.info()

WARNING ⚠️ no model scale passed. Assuming scale='n'.
newconf summary: 512 layers, 2,187,558 parameters, 2,187,542 gradients, 7.2 GFLOPs


(512, 2187558, 2187542, 7.230720000000001)

# !!!Train the model!!!

In [20]:
results = model.train(data="data.yaml", epochs=100, imgsz=640)

Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=ultralytics/ultralytics/cfg/models/v8/newconf.yaml, data=data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=Tr

100%|██████████| 755k/755k [00:00<00:00, 60.4MB/s]


WARNING ⚠️ no model scale passed. Assuming scale='n'.

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  1     58976  ultralytics.nn.modules.block.RepNCSPELAN4    [64, 16, 64, 32, 3]           
  5                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  6                  -1  1    137376  ultralytics.nn.modules.block.RepNCSPELAN4    [32, 64, 128, 48, 3]          
  7                  -1  1     36

100%|██████████| 6.25M/6.25M [00:00<00:00, 320MB/s]


AMP: checks passed ✅


...

    100/100      2.37G     0.6972     0.4567      1.128         22        640: 100%|██████████| 193/193 [01:09<00:00,  2.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:02<00:00,  4.25it/s]

                   all        268        347      0.924       0.85      0.936      0.664



100 epochs completed in 2.128 hours.
Optimizer stripped from /content/ultralytics/runs/detect/train2/weights/last.pt, 4.8MB
Optimizer stripped from /content/ultralytics/runs/detect/train2/weights/best.pt, 4.8MB

Validating /content/ultralytics/runs/detect/train2/weights/best.pt...
Ultralytics YOLOv8.2.90 🚀 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
newconf summary (fused): 297 layers, 2,170,150 parameters, 0 gradients, 7.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:05<00:00,  1.52it/s]


                   all        268        347       0.92      0.841      0.936      0.664
                   FIT        125        162      0.911      0.824      0.934      0.706
                 UNFIT        163        185       0.93      0.859      0.937      0.623
Speed: 0.3ms preprocess, 3.6ms inference, 0.0ms loss, 5.1ms postprocess per image
Results saved to /content/ultralytics/runs/detect/train2


# Download the results

In [22]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [24]:
%pwd

'/content'

In [28]:
# prompt: write a code to download the ultralytics/runs/train2 directory

!zip -r train.zip /content/ultralytics/runs/detect/train2
from google.colab import files
files.download('/content/train.zip')


updating: content/ultralytics/runs/detect/train2/ (stored 0%)
updating: content/ultralytics/runs/detect/train2/val_batch1_pred.jpg (deflated 4%)
updating: content/ultralytics/runs/detect/train2/args.yaml (deflated 53%)
updating: content/ultralytics/runs/detect/train2/results.csv (deflated 85%)
updating: content/ultralytics/runs/detect/train2/events.out.tfevents.1725797970.5643c1dbad27.182.0 (deflated 91%)
updating: content/ultralytics/runs/detect/train2/PR_curve.png (deflated 19%)
updating: content/ultralytics/runs/detect/train2/train_batch17370.jpg (deflated 3%)
updating: content/ultralytics/runs/detect/train2/val_batch1_labels.jpg (deflated 5%)
updating: content/ultralytics/runs/detect/train2/val_batch2_labels.jpg (deflated 4%)
updating: content/ultralytics/runs/detect/train2/train_batch17372.jpg (deflated 2%)
updating: content/ultralytics/runs/detect/train2/R_curve.png (deflated 13%)
updating: content/ultralytics/runs/detect/train2/labels_correlogram.jpg (deflated 41%)
updating: con

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>